#### 2021.05.08 重新进行预处理以及回归预测

In [1]:
# Load train data
import sys
import pandas as pd
import numpy as np

data_file = './data.csv'  # 学生1-6年的近视数据
data = pd.read_csv(data_file, encoding ='gbk')# 有中文用gbk，字符集用utf-8
data.head(10)

,编号,性别,眼别,年龄,1 logMAR,2 logMAR,3 logMAR,4 logMAR,5 logMAR,6 logMAR,...,3eye,4eye,5eye,6eye,1 0.5(K1+K2),2 0.5(K1+K2),3 0.5(K1+K2),4 0.5(K1+K2),5 0.5(K1+K2),6 0.5(K1+K2)
0,1,男,R,7.0,0.000000,0.00000,0.096910,0.60206,0.698970,0.698970,...,24.02,24.82,25.21,25.54,41.625,42.250,41.875,42.000,41.750,42.000
1,2,男,R,8.0,0.000000,0.09691,0.096910,0.00000,0.000000,-0.079181,...,22.12,22.15,23.06,22.30,43.250,43.375,43.125,43.250,43.375,43.750
2,3,男,R,8.0,0.221849,0.30103,1.000000,1.09691,1.000000,0.823909,...,24.90,25.42,25.70,26.31,42.625,42.875,43.125,42.750,42.750,42.875
3,4,男,R,8.0,0.000000,0.00000,0.221849,0.39794,0.096910,0.522879,...,24.23,24.77,25.11,25.47,41.375,41.375,41.750,41.625,41.750,41.750
4,5,男,R,8.0,0.096910,0.00000,0.000000,0.00000,0.154902,0.221849,...,23.26,23.67,23.89,24.33,44.125,47.000,45.000,44.125,44.000,44.375
5,6,男,R,9.0,0.096910,0.00000,0.000000,0.00000,0.397940,0.698970,...,24.09,24.70,25.13,25.74,41.000,41.750,42.375,41.250,41.000,41.500
6,7,男,R,8.0,0.000000,0.00000,0.000000,0.09691,0.301030,0.522879,...,23.53,24.03,24.70,25.32,42.750,43.125,43.250,42.875,42.875,43.375
7,8,男,R,8.0,0.000000,0.00000,0.000000,0.00000,0.301030,0.301030,...,23.77,24.19,24.97,25.00,41.750,41.875,40.125,41.625,41.500,42.000
8,9,男,R,8.0,0.221849,0.30103,0.397940,0.69897,0.522879,1.301030,...,23.90,24.18,23.67,24.64,45.000,45.000,44.875,44.875,44.875,44.875
9,10,男,R,7.0,0.000000,0.00000,0.000000,0.30103,0.301030,0.301030,...,23.33,23.91,23.99,24.59,43.250,48.875,46.500,43.250,43.250,43.375


### Preprocessing
* 对现有数据做预处理，将每一年的数据整合在一块
* 将前面没用的数据不予读取

In [2]:
# 丢弃前4列的数据，切勿重复执行，会一直裁剪数据
data = data.iloc[:, 4:]# 前的：代表行，后面的代表列，：前后是代表从几行（列）到几行（列）
data.head(10)

,1 logMAR,2 logMAR,3 logMAR,4 logMAR,5 logMAR,6 logMAR,1SE,2SE,3SE,4SE,...,3eye,4eye,5eye,6eye,1 0.5(K1+K2),2 0.5(K1+K2),3 0.5(K1+K2),4 0.5(K1+K2),5 0.5(K1+K2),6 0.5(K1+K2)
0,0.000000,0.00000,0.096910,0.60206,0.698970,0.698970,0.625,-0.875,-1.750,-2.500,...,24.02,24.82,25.21,25.54,41.625,42.250,41.875,42.000,41.750,42.000
1,0.000000,0.09691,0.096910,0.00000,0.000000,-0.079181,3.250,2.750,2.625,2.750,...,22.12,22.15,23.06,22.30,43.250,43.375,43.125,43.250,43.375,43.750
2,0.221849,0.30103,1.000000,1.09691,1.000000,0.823909,-0.250,-1.750,-2.500,-4.500,...,24.90,25.42,25.70,26.31,42.625,42.875,43.125,42.750,42.750,42.875
3,0.000000,0.00000,0.221849,0.39794,0.096910,0.522879,0.125,0.125,0.000,-1.000,...,24.23,24.77,25.11,25.47,41.375,41.375,41.750,41.625,41.750,41.750
4,0.096910,0.00000,0.000000,0.00000,0.154902,0.221849,-0.625,-0.250,0.750,-1.250,...,23.26,23.67,23.89,24.33,44.125,47.000,45.000,44.125,44.000,44.375
5,0.096910,0.00000,0.000000,0.00000,0.397940,0.698970,-1.125,0.000,0.625,-0.750,...,24.09,24.70,25.13,25.74,41.000,41.750,42.375,41.250,41.000,41.500
6,0.000000,0.00000,0.000000,0.09691,0.301030,0.522879,-0.250,-1.000,-0.250,-0.250,...,23.53,24.03,24.70,25.32,42.750,43.125,43.250,42.875,42.875,43.375
7,0.000000,0.00000,0.000000,0.00000,0.301030,0.301030,0.250,0.125,0.500,-0.750,...,23.77,24.19,24.97,25.00,41.750,41.875,40.125,41.625,41.500,42.000
8,0.221849,0.30103,0.397940,0.69897,0.522879,1.301030,0.250,-2.000,-3.000,-3.750,...,23.90,24.18,23.67,24.64,45.000,45.000,44.875,44.875,44.875,44.875
9,0.000000,0.00000,0.000000,0.30103,0.301030,0.301030,-0.375,0.125,-0.125,-1.625,...,23.33,23.91,23.99,24.59,43.250,48.875,46.500,43.250,43.250,43.375


In [3]:
# 提取每一年的数据重新排列放入train_data.csv文件中
data1 = data.loc[:,['1SE','1eye','1 0.5(K1+K2)','1 logMAR','2SE','2eye','2 0.5(K1+K2)','2 logMAR'\
                    ,'3SE','3eye','3 0.5(K1+K2)','3 logMAR','4SE','4eye','4 0.5(K1+K2)','4 logMAR'\
                   ,'5SE','5eye','5 0.5(K1+K2)','5 logMAR','6SE','6eye','6 0.5(K1+K2)','6 logMAR']]
data1

,1SE,1eye,1 0.5(K1+K2),1 logMAR,2SE,2eye,2 0.5(K1+K2),2 logMAR,3SE,3eye,...,4 0.5(K1+K2),4 logMAR,5SE,5eye,5 0.5(K1+K2),5 logMAR,6SE,6eye,6 0.5(K1+K2),6 logMAR
0,0.625,23.27,41.625,0.000000,-0.875,23.75,42.250,0.00000,-1.750,24.02,...,42.000,0.602060,-3.125,25.21,41.750,0.698970,-4.250,25.54,42.000,0.698970
1,3.250,22.12,43.250,0.000000,2.750,22.11,43.375,0.09691,2.625,22.12,...,43.250,0.000000,3.375,23.06,43.375,0.000000,2.875,22.30,43.750,-0.079181
2,-0.250,23.45,42.625,0.221849,-1.750,24.29,42.875,0.30103,-2.500,24.90,...,42.750,1.096910,-5.250,25.70,42.750,1.000000,-6.000,26.31,42.875,0.823909
3,0.125,23.59,41.375,0.000000,0.125,23.86,41.375,0.00000,0.000,24.23,...,41.625,0.397940,-2.250,25.11,41.750,0.096910,-2.750,25.47,41.750,0.522879
4,-0.625,22.77,44.125,0.096910,-0.250,22.95,47.000,0.00000,0.750,23.26,...,44.125,0.000000,-1.250,23.89,44.000,0.154902,-1.750,24.33,44.375,0.221849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,-0.750,21.84,45.375,0.000000,0.000,22.70,45.125,0.00000,-0.125,22.70,...,45.250,0.301030,-2.375,23.22,45.250,0.522879,-3.375,23.91,45.375,0.301030
62,0.125,21.87,45.500,0.000000,0.000,22.44,45.625,0.00000,-0.500,22.90,...,45.250,0.522879,-3.250,24.89,45.500,0.522879,-4.000,24.25,45.750,0.522879
63,-3.250,21.90,43.625,0.000000,-0.750,22.23,44.000,0.09691,-0.250,22.53,...,43.625,0.221849,-0.750,23.04,43.750,0.096910,-1.250,23.38,44.000,-0.176091
64,-0.500,21.30,45.500,0.096910,-0.125,21.51,45.375,0.09691,-0.750,21.69,...,45.375,0.000000,0.375,22.49,45.125,0.000000,-0.250,22.32,45.375,0.000000


In [4]:
data1.to_csv('./train_data.csv')

In [5]:
train_data = pd.read_csv('./train_data.csv')
train_data

,Unnamed: 0,1SE,1eye,1 0.5(K1+K2),1 logMAR,2SE,2eye,2 0.5(K1+K2),2 logMAR,3SE,...,4 0.5(K1+K2),4 logMAR,5SE,5eye,5 0.5(K1+K2),5 logMAR,6SE,6eye,6 0.5(K1+K2),6 logMAR
0,0,0.625,23.27,41.625,0.000000,-0.875,23.75,42.250,0.00000,-1.750,...,42.000,0.602060,-3.125,25.21,41.750,0.698970,-4.250,25.54,42.000,0.698970
1,1,3.250,22.12,43.250,0.000000,2.750,22.11,43.375,0.09691,2.625,...,43.250,0.000000,3.375,23.06,43.375,0.000000,2.875,22.30,43.750,-0.079181
2,2,-0.250,23.45,42.625,0.221849,-1.750,24.29,42.875,0.30103,-2.500,...,42.750,1.096910,-5.250,25.70,42.750,1.000000,-6.000,26.31,42.875,0.823909
3,3,0.125,23.59,41.375,0.000000,0.125,23.86,41.375,0.00000,0.000,...,41.625,0.397940,-2.250,25.11,41.750,0.096910,-2.750,25.47,41.750,0.522879
4,4,-0.625,22.77,44.125,0.096910,-0.250,22.95,47.000,0.00000,0.750,...,44.125,0.000000,-1.250,23.89,44.000,0.154902,-1.750,24.33,44.375,0.221849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,-0.750,21.84,45.375,0.000000,0.000,22.70,45.125,0.00000,-0.125,...,45.250,0.301030,-2.375,23.22,45.250,0.522879,-3.375,23.91,45.375,0.301030
62,62,0.125,21.87,45.500,0.000000,0.000,22.44,45.625,0.00000,-0.500,...,45.250,0.522879,-3.250,24.89,45.500,0.522879,-4.000,24.25,45.750,0.522879
63,63,-3.250,21.90,43.625,0.000000,-0.750,22.23,44.000,0.09691,-0.250,...,43.625,0.221849,-0.750,23.04,43.750,0.096910,-1.250,23.38,44.000,-0.176091
64,64,-0.500,21.30,45.500,0.096910,-0.125,21.51,45.375,0.09691,-0.750,...,45.375,0.000000,0.375,22.49,45.125,0.000000,-0.250,22.32,45.375,0.000000


In [6]:
# 丢弃前1列的数据，切勿重复执行，会一直裁剪数据
train_data = train_data.iloc[:, 1:]
train_data

,1SE,1eye,1 0.5(K1+K2),1 logMAR,2SE,2eye,2 0.5(K1+K2),2 logMAR,3SE,3eye,...,4 0.5(K1+K2),4 logMAR,5SE,5eye,5 0.5(K1+K2),5 logMAR,6SE,6eye,6 0.5(K1+K2),6 logMAR
0,0.625,23.27,41.625,0.000000,-0.875,23.75,42.250,0.00000,-1.750,24.02,...,42.000,0.602060,-3.125,25.21,41.750,0.698970,-4.250,25.54,42.000,0.698970
1,3.250,22.12,43.250,0.000000,2.750,22.11,43.375,0.09691,2.625,22.12,...,43.250,0.000000,3.375,23.06,43.375,0.000000,2.875,22.30,43.750,-0.079181
2,-0.250,23.45,42.625,0.221849,-1.750,24.29,42.875,0.30103,-2.500,24.90,...,42.750,1.096910,-5.250,25.70,42.750,1.000000,-6.000,26.31,42.875,0.823909
3,0.125,23.59,41.375,0.000000,0.125,23.86,41.375,0.00000,0.000,24.23,...,41.625,0.397940,-2.250,25.11,41.750,0.096910,-2.750,25.47,41.750,0.522879
4,-0.625,22.77,44.125,0.096910,-0.250,22.95,47.000,0.00000,0.750,23.26,...,44.125,0.000000,-1.250,23.89,44.000,0.154902,-1.750,24.33,44.375,0.221849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,-0.750,21.84,45.375,0.000000,0.000,22.70,45.125,0.00000,-0.125,22.70,...,45.250,0.301030,-2.375,23.22,45.250,0.522879,-3.375,23.91,45.375,0.301030
62,0.125,21.87,45.500,0.000000,0.000,22.44,45.625,0.00000,-0.500,22.90,...,45.250,0.522879,-3.250,24.89,45.500,0.522879,-4.000,24.25,45.750,0.522879
63,-3.250,21.90,43.625,0.000000,-0.750,22.23,44.000,0.09691,-0.250,22.53,...,43.625,0.221849,-0.750,23.04,43.750,0.096910,-1.250,23.38,44.000,-0.176091
64,-0.500,21.30,45.500,0.096910,-0.125,21.51,45.375,0.09691,-0.750,21.69,...,45.375,0.000000,0.375,22.49,45.125,0.000000,-0.250,22.32,45.375,0.000000


In [7]:
# 把dataframe转换成numpy数组，此时是65*24
train_data = train_data.to_numpy()
train_data

array([[ 0.625     , 23.27      , 41.625     , ..., 25.54      ,
        42.        ,  0.69897   ],
       [ 3.25      , 22.12      , 43.25      , ..., 22.3       ,
        43.75      , -0.07918125],
       [-0.25      , 23.45      , 42.625     , ..., 26.31      ,
        42.875     ,  0.82390874],
       ...,
       [-3.25      , 21.9       , 43.625     , ..., 23.38      ,
        44.        , -0.17609126],
       [-0.5       , 21.3       , 45.5       , ..., 22.32      ,
        45.375     ,  0.        ],
       [-1.        , 22.33      , 42.875     , ..., 23.09      ,
        44.25      ,  0.        ]])

#### 分离数据
* 将前五年的数据和第六年的近视数据分离出来

#### 2021.05.08 提取特征（Extract Feature）
* 把65 * 20的数据放入x，即前五年的数据
* 把65 * 1的数据放入y，即标签数据

In [8]:
x = np.empty([65, 20], dtype = float)
y = np.empty([65, 1], dtype = float)
for people in range(65):
    x[people, :] = train_data[people, 0:20]
    y[people, :] = train_data[people, 23]
print(x)
print(y)

[[ 0.625      23.27       41.625      ... 25.21       41.75
   0.69897   ]
 [ 3.25       22.12       43.25       ... 23.06       43.375
   0.        ]
 [-0.25       23.45       42.625      ... 25.7        42.75
   1.        ]
 ...
 [ 0.125      21.87       45.5        ... 24.89       45.5
   0.52287874]
 [-3.25       21.9        43.625      ... 23.04       43.75
   0.09691001]
 [-0.5        21.3        45.5        ... 22.49       45.125
   0.        ]]
[[ 0.69897   ]
 [-0.07918125]
 [ 0.82390874]
 [ 0.52287874]
 [ 0.22184875]
 [ 0.69897   ]
 [ 0.52287874]
 [ 0.30103   ]
 [ 1.30103   ]
 [ 0.30103   ]
 [ 0.22184875]
 [ 0.22184875]
 [ 0.60205999]
 [ 0.82390874]
 [ 0.82390874]
 [ 0.22184875]
 [ 0.60205999]
 [ 0.39794001]
 [ 0.09691001]
 [ 0.        ]
 [ 0.        ]
 [ 0.69897   ]
 [ 0.22184875]
 [ 0.22184875]
 [ 0.69897   ]
 [ 0.69897   ]
 [ 0.        ]
 [ 0.30103   ]
 [ 0.        ]
 [ 0.30103   ]
 [ 0.60205999]
 [ 0.69897   ]
 [ 0.30103   ]
 [ 0.        ]
 [ 0.60205999]
 [ 0.30103   ]
 [ 

#### 标准化（Normalization）
* Normalization和Standardization分别被翻译成归一化和标准化，其实是有问题的，很容易被混淆。实际上，两者可以统称为标准化（Normalization）
 * x = (x − μ)/σ 叫做z-score normalization
 * x = (x − xmin) / (xmax − xmin)又叫做min-max normalization，网上的某些资料有点问题。
 * x = (x − μ) / σ，μ是x的均值，σ是x的标准差。


* 通过标准化，可以：
 * 将有量纲的表达式，经过变换，化为无量纲的表达式，成为标量
 * 使得数据更加符合独立同分布条件
 
 
* 这个转换使得x的均值为0，标准差为1，而不是像网上说的变成(-1,1)之间的数据。从下方的out中，可以看到有部分数据是小于-1的。


* 这里每一列是一个观测指标，按列进行标准化。

 将数据标准化

In [9]:
len(x)

65

In [10]:
len(x[0])

20

In [11]:
#标准化  # 求平均
mean_x = np.mean(x, axis = 0) # 计算每一列的平均值（axis = 0），axis = 1则求每一行的平均值
std_x = np.std(x, axis = 0) # 计算标准差
print(mean_x)
print(std_x)

[-3.07692308e-02  2.26664615e+01  4.29598462e+01  5.70813147e-02
 -3.57692308e-01  2.30992308e+01  4.35403846e+01  7.64996773e-02
 -6.92307692e-02  2.34218462e+01  4.34269231e+01  1.80107782e-01
 -1.32115385e+00  2.38563077e+01  4.30992308e+01  2.61003827e-01
 -1.87115385e+00  2.40290769e+01  4.30623077e+01  3.27631709e-01]
[1.07619137 0.74899611 1.21313991 0.08877158 1.04740451 0.81256537
 1.69616475 0.17992107 1.33769872 0.9116239  1.39692181 0.24213624
 1.59053227 0.99927908 1.20858284 0.31164009 1.77968899 0.93721637
 1.2077086  0.3244766 ]


### Note：

###### 标准化的一些概念
* std_x = np.std(x, axis = 0, ddof=1)
* [-3.07692308e-02  2.26664615e+01  4.29598462e+01  5.70813147e-02
 -3.57692308e-01  2.30992308e+01  4.35403846e+01  7.64996773e-02
 -6.92307692e-02  2.34218462e+01  4.34269231e+01  1.80107782e-01
 -1.32115385e+00  2.38563077e+01  4.30992308e+01  2.61003827e-01
 -1.87115385e+00             nan  4.30623077e+01  3.27631709e-01]
* [1.08456653 0.75482496 1.22258083 0.08946242 1.05555564 0.81888893
 1.70936467 0.18132126 1.34810899 0.91871835 1.40779296 0.2440206
 1.60291014 1.00705569 1.2179883  0.31406534 1.79353892        nan
 1.21710725 0.32700175]
 
* numpy默认是除以样本数，求的是母体标准差；matlab默认是除以样本-1，求的是样本标准差
* 为什么matlab计算出标准差是1，原来有【样本标准差】、【总体标准差】（母体标准差）之分。 
 * 总体标准差：sqrt(sum(每个元素与平均值的差值的平方)/(元素数))；
 * 样本标准差：sqrt(sum(每个元素与平均值的差值的平方)/(元素数-1))；
 * 差别在分母是(元素数)还是(元素数-1)，ddof相当于其中减去的数字1参数，默认为0。
 

##### 标准化后发现有nan值，检查后发现数据有空值，需要操作
##### 5.8晚经过尝试后发现就是空值引起的nan值，需要进行数据的一个处理

##### 5.10 只有一个空值，采取人工填充，此人的SE数据是逐年递增的，找一个类似的人员的数据填充进去

### End

In [12]:
for i in range(len(x)):
    for j in range(len(x[0])):
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

array([[ 0.60934258,  0.80579652, -1.10032334, ...,  1.26003248,
        -1.08660955,  1.1444224 ],
       [ 3.04849984, -0.72959196,  0.23917591, ..., -1.03399487,
         0.2589137 , -1.00972367],
       [-0.20370984,  1.0461182 , -0.27601611, ...,  1.78285732,
        -0.25859524,  2.07216262],
       ...,
       [ 0.1447412 , -1.06337206,  2.09386719, ...,  0.91859586,
         2.01844411,  0.60172916],
       [-2.99131814, -1.02331845,  0.54829113, ..., -1.05533466,
         0.56941907, -0.71105803],
       [-0.43601053, -1.8243907 ,  2.09386719, ..., -1.64217887,
         1.70793874, -1.00972367]])

### Note：
* mean()函数功能：求取均值：https://blog.csdn.net/taotiezhengfeng/article/details/72397282
* 经常操作的参数为axis，以m * n矩阵举例：
 * axis 不设置值，对 mn 个数求均值，返回一个实数
 * axis = 0：压缩行，对各列求均值，返回 1*n 矩阵
 * axis = 1 ：压缩列，对各行求均值，返回 m*1 矩阵
 
 
 
### End

把训练数据分成训练集train_set和验证集validation，其中train_set用于训练，而validation不会参与训练，仅用于验证。（在baseline中并没有用）

In [13]:
# 将训练集分成训练-验证集，用来最后检验我们的模型
# math.floor()是取整，没有四舍五入，直接取整
import math
x_train_set = x[: math.floor(len(x) * 0.8), :]
y_train_set = y[: math.floor(len(y) * 0.8), :]
x_validation = x[math.floor(len(x) * 0.8): , :]
y_validation = y[math.floor(len(y) * 0.8): , :]
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)
print(len(x_train_set))
print(len(y_train_set))
print(len(x_validation))
print(len(y_validation))

[[ 0.60934258  0.80579652 -1.10032334 ...  1.26003248 -1.08660955
   1.1444224 ]
 [ 3.04849984 -0.72959196  0.23917591 ... -1.03399487  0.2589137
  -1.00972367]
 [-0.20370984  1.0461182  -0.27601611 ...  1.78285732 -0.25859524
   2.07216262]
 ...
 [ 0.02859085  0.24504595 -0.99728493 ... -0.25509256 -0.98310776
  -1.00972367]
 [ 0.02859085  0.00472427  0.85740634 ...  0.28907207  0.77642265
   0.21668219]
 [-0.90061191 -0.5426751   0.54829113 ... -0.69255824  0.46591728
  -0.53233345]]
[[ 0.69897   ]
 [-0.07918125]
 [ 0.82390874]
 [ 0.52287874]
 [ 0.22184875]
 [ 0.69897   ]
 [ 0.52287874]
 [ 0.30103   ]
 [ 1.30103   ]
 [ 0.30103   ]
 [ 0.22184875]
 [ 0.22184875]
 [ 0.60205999]
 [ 0.82390874]
 [ 0.82390874]
 [ 0.22184875]
 [ 0.60205999]
 [ 0.39794001]
 [ 0.09691001]
 [ 0.        ]
 [ 0.        ]
 [ 0.69897   ]
 [ 0.22184875]
 [ 0.22184875]
 [ 0.69897   ]
 [ 0.69897   ]
 [ 0.        ]
 [ 0.30103   ]
 [ 0.        ]
 [ 0.30103   ]
 [ 0.60205999]
 [ 0.69897   ]
 [ 0.30103   ]
 [ 0.        ]

##### 训练数据

* Loss的代码用到的是 Root Mean Square Error(均方根误差RMSE)
* 因为存在常数项b，所以维度（dim）需要多加一列；eps项是极小值，避免adagrad的分母为0.
* 每一个维度（dim）会对应到各自的gradient和权重w，通过一次次的迭代（iter_time）学习。最终，将训练得到的模型（权重w）存储为.npy格式的文件。


In [14]:
dim = 20 + 1
w = np.zeros([dim, 1])
x = np.concatenate((np.ones([65, 1]), x), axis = 1).astype(float)
learning_rate = 100
iter_time = 3000
adagrad = np.zeros([dim, 1])
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/65)#rmse
    if(t%100==0):
        print("迭代的次数：%i ， 损失值：%f"%(t,loss))
        #print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / np.sqrt(adagrad + eps)
np.save('weight.npy', w)
w

迭代的次数：0 ， 损失值：0.530321
迭代的次数：100 ， 损失值：12.810135
迭代的次数：200 ， 损失值：8.874932
迭代的次数：300 ， 损失值：6.417246
迭代的次数：400 ， 损失值：4.694935
迭代的次数：500 ， 损失值：3.451632
迭代的次数：600 ， 损失值：2.545824
迭代的次数：700 ， 损失值：1.883361
迭代的次数：800 ， 损失值：1.398016
迭代的次数：900 ， 损失值：1.042359
迭代的次数：1000 ， 损失值：0.782170
迭代的次数：1100 ， 损失值：0.592660
迭代的次数：1200 ， 损失值：0.455809
迭代的次数：1300 ， 损失值：0.358418
迭代的次数：1400 ， 损失值：0.290635
迭代的次数：1500 ， 损失值：0.244858
迭代的次数：1600 ， 损失值：0.215013
迭代的次数：1700 ， 损失值：0.196220
迭代的次数：1800 ， 损失值：0.184722
迭代的次数：1900 ， 损失值：0.177821
迭代的次数：2000 ， 损失值：0.173721
迭代的次数：2100 ， 损失值：0.171290
迭代的次数：2200 ， 损失值：0.169842
迭代的次数：2300 ， 损失值：0.168974
迭代的次数：2400 ， 损失值：0.168447
迭代的次数：2500 ， 损失值：0.168123
迭代的次数：2600 ， 损失值：0.167921
迭代的次数：2700 ， 损失值：0.167794
迭代的次数：2800 ， 损失值：0.167713
迭代的次数：2900 ， 损失值：0.167660


array([[ 0.40530723],
       [ 0.08653884],
       [-0.05221514],
       [-0.06250956],
       [ 0.01150364],
       [-0.03931023],
       [-0.04761889],
       [ 0.02268408],
       [ 0.01399131],
       [-0.053869  ],
       [ 0.15887673],
       [ 0.00883229],
       [-0.0609081 ],
       [ 0.15964754],
       [-0.06574318],
       [ 0.09316924],
       [ 0.05085272],
       [-0.20718467],
       [ 0.02662812],
       [-0.064153  ],
       [ 0.15589723]])

### Note：
* np.zeros()、np.ones().astype(float)
 * 

* np.concatenate()
 * 
 
* loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/65)
 * np.sqrt
 * np.sum
 * np.power
 * np.dot
 
* gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y)
 * x.transpose()
 
 

### End

#### 测试：首先对test.csv进行数据预处理

In [15]:
#读取测试数据
testdata_file = './test.csv'  # 测试学生1-6年的近视数据
testdata = pd.read_csv(testdata_file)# , encoding ='gbk'有中文用gbk，字符集用utf-8
testdata.head(11)

,编号,性别,眼别,年龄,1 logMAR,2 logMAR,3 logMAR,4 logMAR,5 logMAR,6 logMAR,...,3eye,4eye,5eye,6eye,1 0.5(K1+K2),2 0.5(K1+K2),3 0.5(K1+K2),4 0.5(K1+K2),5 0.5(K1+K2),6 0.5(K1+K2)
0,66,女,R,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,22.74,22.86,22.88,23.09,42.875,47.125,42.875,43.000,43.000,44.250
1,67,女,R,7,0.096910,0.397940,0.698970,0.096910,0.096910,0.096910,...,23.98,24.43,24.70,24.94,42.875,44.500,44.375,41.125,41.000,40.625
2,68,女,R,7,0.000000,0.000000,0.000000,0.000000,0.000000,0.096910,...,23.18,23.56,23.86,24.08,40.875,41.250,44.875,41.250,41.125,41.375
3,69,女,R,8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,23.18,23.36,23.61,23.66,41.125,44.250,41.250,41.375,41.125,41.625
4,70,女,R,8,0.221849,0.000000,0.000000,0.000000,0.045757,0.397940,...,23.19,23.73,24.19,24.73,45.000,42.625,42.625,42.500,42.375,42.875
5,71,女,R,8,0.000000,0.221849,0.698970,0.397940,0.301030,0.920819,...,23.83,22.40,24.82,24.98,43.875,44.250,47.625,44.000,44.125,44.375
6,72,女,R,7,0.000000,0.000000,0.000000,0.000000,0.397940,0.698970,...,23.16,23.55,24.19,24.67,43.375,42.250,43.625,43.625,43.625,43.750
7,73,女,R,7,0.000000,0.000000,0.000000,0.000000,0.522879,0.920819,...,23.05,23.82,24.69,23.55,42.625,42.625,42.750,42.625,42.625,43.000
8,74,女,R,7,0.096910,0.301030,0.602060,0.301030,0.397940,0.522879,...,25.28,25.89,26.35,24.07,43.125,43.625,46.500,43.375,43.250,43.625
9,75,女,R,8,0.000000,0.000000,0.000000,0.602060,1.000000,0.221849,...,24.00,24.47,24.97,25.45,41.625,41.750,42.000,41.750,41.750,42.125


In [16]:
# 丢弃前4列的数据，切勿重复执行，会一直裁剪数据
testdata = testdata.iloc[:, 4:]# 前的：代表行，后面的代表列，：前后是代表从几行（列）到几行（列）
testdata.head(10)

,1 logMAR,2 logMAR,3 logMAR,4 logMAR,5 logMAR,6 logMAR,1SE,2SE,3SE,4SE,...,3eye,4eye,5eye,6eye,1 0.5(K1+K2),2 0.5(K1+K2),3 0.5(K1+K2),4 0.5(K1+K2),5 0.5(K1+K2),6 0.5(K1+K2)
0,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,-1.00,-1.750,-0.625,0.000,...,22.74,22.86,22.88,23.09,42.875,47.125,42.875,43.000,43.000,44.250
1,0.096910,0.397940,0.69897,0.09691,0.096910,0.096910,0.50,-1.625,-3.000,-1.000,...,23.98,24.43,24.70,24.94,42.875,44.500,44.375,41.125,41.000,40.625
2,0.000000,0.000000,0.00000,0.00000,0.000000,0.096910,0.00,0.250,0.750,-0.750,...,23.18,23.56,23.86,24.08,40.875,41.250,44.875,41.250,41.125,41.375
3,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00,0.375,0.250,0.000,...,23.18,23.36,23.61,23.66,41.125,44.250,41.250,41.375,41.125,41.625
4,0.221849,0.000000,0.00000,0.00000,0.045757,0.397940,1.75,0.500,0.375,-2.000,...,23.19,23.73,24.19,24.73,45.000,42.625,42.625,42.500,42.375,42.875
5,0.000000,0.221849,0.69897,0.39794,0.301030,0.920819,-0.75,-1.500,-1.500,-3.500,...,23.83,22.40,24.82,24.98,43.875,44.250,47.625,44.000,44.125,44.375
6,0.000000,0.000000,0.00000,0.00000,0.397940,0.698970,0.00,0.500,1.000,0.000,...,23.16,23.55,24.19,24.67,43.375,42.250,43.625,43.625,43.625,43.750
7,0.000000,0.000000,0.00000,0.00000,0.522879,0.920819,1.00,1.000,1.250,-0.625,...,23.05,23.82,24.69,23.55,42.625,42.625,42.750,42.625,42.625,43.000
8,0.096910,0.301030,0.60206,0.30103,0.397940,0.522879,-1.00,-5.250,-5.250,-7.875,...,25.28,25.89,26.35,24.07,43.125,43.625,46.500,43.375,43.250,43.625
9,0.000000,0.000000,0.00000,0.60206,1.000000,0.221849,0.50,0.250,0.500,-0.750,...,24.00,24.47,24.97,25.45,41.625,41.750,42.000,41.750,41.750,42.125


In [17]:
# 提取每一年的数据重新排列
testdata1 = testdata.loc[:,['1SE','1eye','1 0.5(K1+K2)','1 logMAR','2SE','2eye','2 0.5(K1+K2)','2 logMAR'\
                    ,'3SE','3eye','3 0.5(K1+K2)','3 logMAR','4SE','4eye','4 0.5(K1+K2)','4 logMAR'\
                   ,'5SE','5eye','5 0.5(K1+K2)','5 logMAR','6SE','6eye','6 0.5(K1+K2)','6 logMAR']]
testdata1

,1SE,1eye,1 0.5(K1+K2),1 logMAR,2SE,2eye,2 0.5(K1+K2),2 logMAR,3SE,3eye,...,4 0.5(K1+K2),4 logMAR,5SE,5eye,5 0.5(K1+K2),5 logMAR,6SE,6eye,6 0.5(K1+K2),6 logMAR
0,-1.000,22.33,42.875,0.000000,-1.750,22.54,47.125,0.000000,-0.625,22.74,...,43.000,0.000000,-0.125,22.88,43.000,0.000000,-0.250,23.09,44.250,0.000000
1,0.500,22.38,42.875,0.096910,-1.625,23.51,44.500,0.397940,-3.000,23.98,...,41.125,0.096910,-1.875,24.70,41.000,0.096910,-3.375,24.94,40.625,0.096910
2,0.000,22.70,40.875,0.000000,0.250,22.95,41.250,0.000000,0.750,23.18,...,41.250,0.000000,-0.625,23.86,41.125,0.000000,-1.750,24.08,41.375,0.096910
3,0.000,22.93,41.125,0.000000,0.375,23.04,44.250,0.000000,0.250,23.18,...,41.375,0.000000,0.250,23.61,41.125,0.000000,-0.750,23.66,41.625,0.000000
4,1.750,22.48,45.000,0.221849,0.500,22.84,42.625,0.000000,0.375,23.19,...,42.500,0.000000,-1.250,24.19,42.375,0.045757,-2.375,24.73,42.875,0.397940
5,-0.750,22.45,43.875,0.000000,-1.500,23.36,44.250,0.221849,-1.500,23.83,...,44.000,0.397940,-4.375,24.82,44.125,0.301030,-4.625,24.98,44.375,0.920819
6,0.000,22.46,43.375,0.000000,0.500,22.78,42.250,0.000000,1.000,23.16,...,43.625,0.000000,-1.250,24.19,43.625,0.397940,-2.500,24.67,43.750,0.698970
7,1.000,22.38,42.625,0.000000,1.000,22.74,42.625,0.000000,1.250,23.05,...,42.625,0.000000,-2.500,24.69,42.625,0.522879,-3.750,23.55,43.000,0.920819
8,-1.000,23.50,43.125,0.096910,-5.250,24.75,43.625,0.301030,-5.250,25.28,...,43.375,0.301030,-7.625,26.35,43.250,0.397940,-12.000,24.07,43.625,0.522879
9,0.500,23.42,41.625,0.000000,0.250,23.85,41.750,0.000000,0.500,24.00,...,41.750,0.602060,-1.375,24.97,41.750,1.000000,-2.750,25.45,42.125,0.221849


In [18]:
# 将dataframe变成numpy数组
test_data = testdata1
test_data = test_data.to_numpy()
# 将test数据也变成和训练数据一样的形式，取前五年的数据
test_x = np.empty([10, 20], dtype = float)
for t_people in range(10):
    test_x[t_people, :] = train_data[t_people, 0:20]
test_x

array([[ 0.625     , 23.27      , 41.625     ,  0.        , -0.875     ,
        23.75      , 42.25      ,  0.        , -1.75      , 24.02      ,
        41.875     ,  0.09691001, -2.5       , 24.82      , 42.        ,
         0.60205999, -3.125     , 25.21      , 41.75      ,  0.69897   ],
       [ 3.25      , 22.12      , 43.25      ,  0.        ,  2.75      ,
        22.11      , 43.375     ,  0.09691001,  2.625     , 22.12      ,
        43.125     ,  0.09691001,  2.75      , 22.15      , 43.25      ,
         0.        ,  3.375     , 23.06      , 43.375     ,  0.        ],
       [-0.25      , 23.45      , 42.625     ,  0.22184875, -1.75      ,
        24.29      , 42.875     ,  0.30103   , -2.5       , 24.9       ,
        43.125     ,  1.        , -4.5       , 25.42      , 42.75      ,
         1.09691001, -5.25      , 25.7       , 42.75      ,  1.        ],
       [ 0.125     , 23.59      , 41.375     ,  0.        ,  0.125     ,
        23.86      , 41.375     ,  0.        ,  

In [19]:
# 数据标准化  
for i in range(len(test_x)):
    for j in range(len(test_x[0])):
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
#扩展出bais
test_x = np.concatenate((np.ones([10, 1]), test_x), axis = 1).astype(float)
test_x

array([[ 1.00000000e+00,  6.09342583e-01,  8.05796524e-01,
        -1.10032334e+00, -6.43013366e-01, -4.93894848e-01,
         8.00882307e-01, -7.60766085e-01, -4.25184644e-01,
        -1.25646321e+00,  6.56141033e-01, -1.11095916e+00,
        -3.43598991e-01, -7.41164563e-01,  9.64387556e-01,
        -9.09520416e-01,  1.09439116e+00, -7.04531051e-01,
         1.26003248e+00, -1.08660955e+00,  1.14442240e+00],
       [ 1.00000000e+00,  3.04849984e+00, -7.29591959e-01,
         2.39175914e-01, -6.43013366e-01,  2.96704117e+00,
        -1.21741684e+00, -9.75050422e-02,  1.13440496e-01,
         2.01407890e+00, -1.42805181e+00, -2.16134558e-01,
        -3.43598991e-01,  2.55961726e+00, -1.70753870e+00,
         1.24748776e-01, -8.37516845e-01,  2.94779249e+00,
        -1.03399487e+00,  2.58913705e-01, -1.00972367e+00],
       [ 1.00000000e+00, -2.03709837e-01,  1.04611820e+00,
        -2.76016106e-01,  1.85608309e+00, -1.32929320e+00,
         1.46544422e+00, -3.92287728e-01,  1.24793786e

载入模型即可对test数据进行预测，得到预测值ans_y。

In [20]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)
ans_y

array([[ 0.83529373],
       [-0.01314604],
       [ 0.97803455],
       [ 0.36155377],
       [ 0.18132986],
       [ 0.42899141],
       [ 0.4939779 ],
       [ 0.38415968],
       [ 0.81534015],
       [ 0.49998665]])

### Note：
*  结论：结果基本不正确
* 6 logMAR
 * 0
 * 0.096910013
 * 0.096910013
 * 0
 * 0.397940009
 * 0.920818754
 * 0.698970004
 * 0.920818754
 * 0.522878745
 * 0.22184875
 * 0.920818754


* 迭代2000次
array([[ 0.8710959 ],
       [-0.06579503],
       [ 0.95531462],
       [ 0.32762571],
       [ 0.17999072],
       [ 0.46727051],
       [ 0.4753372 ],
       [ 0.40717004],
       [ 0.86529623],
       [ 0.50448265]])
       
       
* 迭代1000次
array([[ 1.9379924 ],
       [-1.13685684],
       [ 0.41680339],
       [ 0.06290571],
       [ 0.22798441],
       [ 1.37259662],
       [ 0.50592258],
       [ 0.697144  ],
       [ 1.11135342],
       [ 0.74287458]])
       
       
#### 需要思考如何解决

### End